In [1]:
%%html
<a href=notebook1.ipynb target="_blank">Start a new query!</a>

In [2]:
import sys
import tabulate
import requests
from datetime import date
from random import randint
import numpy as np
from numpy  import array
import pandas as pd
import math

from metrics.api.getEditorInfo import getEditorInfo
from datetime import datetime

from IPython.display import Markdown as md
from IPython.display import display, HTML, clear_output
from ipywidgets import *
from ipywidgets import widgets, interact,Button, HBox, VBox

from bokeh.io import show, output_notebook
from bokeh.layouts import widgetbox
from bokeh.models import ColumnDataSource
from bokeh.models.widgets import DataTable, DateFormatter, TableColumn
from bokeh.models.widgets import CheckboxGroup
from bokeh.models.widgets import Panel, Tabs
from bokeh.plotting import *

import plotly
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.graph_objs import *

plotly.tools.set_credentials_file(username='fkramer', api_key='LmCFXvemFzCfOyuLsP9u')
plotly.tools.set_config_file(world_readable=True,sharing='public')

%store -r userid_input
%store -r username_input

userid_input = int(userid_input)

ModuleNotFoundError: No module named 'plotly'

In [ ]:
md('# User-related information for the wikipedia user: %s and all contributed articles'%username_input)

### Network Statistics

In [ ]:
result = getEditorInfo(editor_id=userid_input)
regday = list(result.values())[3]

def d(regday):
    date_format = "%Y-%m-%d"
    aa = regday[:10]

    a = datetime.strptime(aa, date_format)
    b = datetime.today()
    b = datetime.strptime(str(b)[:10],date_format)
    delta = b - a
    return delta.days

def ymd(regday):
    regdayy = d(regday)
    year = regdayy / 365.25
    month = (regdayy - (int(year)*365.25))/30
    #monthh = round(month, 1)
    day = (regdayy-((regdayy - (int(year)*365.25)) + (regdayy - (int(month)*30))))*-1
    
    line = '%i y %i m %i d'% (int(year),int(month),day)
    return line

if ymd(regday) == '0 y 0 m 0 d':
    editD = 0
else:
    editD = round(list(result.values())[2]/d(regday),4)


table = [
    ['Username: ',username_input],
    ['Userid: ',userid_input],
    ['Gender: ',list(result.values())[4]],
    ['Registration Date: ',regday],
    ['Days registered: ',ymd(regday)],
    ['Editcount: ',list(result.values())[2]],
    ['Edits per day: ',editD]
       ]

display(HTML(tabulate.tabulate(table, tablefmt='html')))

username2 = username_input.replace(" ", "_")
print('Link to Userpage: https://en.wikipedia.org/wiki/User:%s' %(username2))

### Articles Summary

In [ ]:
md('#### Number of article pages user %s has created, deleted, redirected'%username_input)

In [ ]:
pages = requests.get('https://xtools.wmflabs.org/api/user/pages_count/en.wikipedia/%s'% username2).json()

try:
    if (len(pages['counts'])!=0):
        table = [
            ['Pages created: ',pages['counts']['count']],
            ['Pages deleted: ',pages['counts']['deleted']],
            ['Page redirects: ',pages['counts']['redirects']]
               ]

        display(HTML(tabulate.tabulate(table, tablefmt='html')))
    else:
        table = [
            ['Pages created: ',0],
            ['Pages deleted: ',0],
            ['Page redirects: ',0]
               ]

        display(HTML(tabulate.tabulate(table, tablefmt='html')))
except TypeError:
    print("No data available for User "+str(username_input))
    pass

In [ ]:
md('#### Article pages created by user %s '%username_input)

In [ ]:
page_statistic = requests.get('https://xtools.wmflabs.org/api/user/pages/en.wikipedia/%s' % username_input).json()

df_page = pd.DataFrame()


if (len(page_statistic['pages']) !=0):
    for i in range (0,len(page_statistic['pages'])):
        page_title = page_statistic['pages'][i]['page_title']
        page_len = page_statistic['pages'][i]['page_len'] if page_statistic['pages'][i]['page_len'] is not None else "No information available"
        creation = page_statistic['pages'][i]['human_time']
        df_page = df_page.append({'Page Title':page_title, 'Page Length':page_len, 'Creation Date':creation}, ignore_index=True)
    
    source = ColumnDataSource(df_page)

    columns = [
            TableColumn(field="Page Title", title='Talk Page Title'),
            TableColumn(field="Page Length", title="Page Length"),
            TableColumn(field="Creation Date", title="Creation Date"),
         
        ]
    data_table = DataTable(source=source, columns=columns, width=900, height=400)

    output_notebook()

    show(widgetbox(data_table))
    
else:
    print('No Article pages have been created by User '+str(username_input))

#print(df_page)


### Talkpages Summary

In [ ]:
md('#### Number of talk pages user %s has created, deleted, redirected'%username_input)

In [ ]:
talkpages = requests.get('https://xtools.wmflabs.org/api/user/pages_count/en.wikipedia/%s/1/onlyredirects' % username2).json()

try:
    if (len(talkpages['counts']) !=0):
        table = [
            ['Talkpages created: ',talkpages['counts']['count']],
            ['Talkpages deleted: ',talkpages['counts']['deleted']],
            ['Talkpages redirects: ',talkpages['counts']['redirects']]
               ]

        display(HTML(tabulate.tabulate(table, tablefmt='html')))
    else:
        table = [
            ['Talkpages created: ',0],
            ['Talkpages deleted: ',0],
            ['Talkpages redirects: ',0]
               ]

        display(HTML(tabulate.tabulate(table, tablefmt='html')))
except TypeError:
    print("No data available for User "+str(username_input))
    pass

In [ ]:
md('#### Talk pages created by %s '%username_input)

In [ ]:
talkpage_statistic = requests.get('https://xtools.wmflabs.org/api/user/pages/en.wikipedia/%s/1/onlyredirects' % username_input).json()

df_talk = pd.DataFrame()

if ((len(talkpage_statistic['pages'])) != 0):
    for i in range (0,len(talkpage_statistic['pages'])):
        page_title = talkpage_statistic['pages'][i]['page_title']
        page_len = talkpage_statistic['pages'][i]['page_len'] if talkpage_statistic['pages'][i]['page_len'] is not None else "No information available"
        creation = talkpage_statistic['pages'][i]['human_time']
        df_talk = df_talk.append({'Talk Page Title':page_title, 'Page Length': page_len, 'Creation Date':creation}, ignore_index=True)

    source = ColumnDataSource(df_talk)

    columns = [
            TableColumn(field="Talk Page Title", title='Talk Page Title'),
            TableColumn(field="Page Length", title="Page Length"),
            TableColumn(field="Creation Date", title="Creation Date"),

        ]
    data_table = DataTable(source=source, columns=columns, width=900, height=400)

    output_notebook()

    show(widgetbox(data_table))

else:
    print('No article pages have been created by User '+str(username_input))

### User Activity - ungrouped

In [ ]:
import os.path, time
from metrics.getEditorActions import getEditorActions

feditorActions = 'cache/%i_%s_editorActions.csv' % (userid_input, username_input)
fwordsOwned = 'cache/%i_%s_wordsOwned.csv' % (userid_input, username_input)

#Checks if there are records in the cache folder.

if os.path.isfile(feditorActions)== True & os.path.isfile(fwordsOwned) == True:
    
    print('A dataset for user '+str(username_input)+' already exists in the cache. Do you want to use it?')
    print('Otherwise you can start a new query.')
    print("last modified: %s \n" % time.ctime(os.path.getmtime(feditorActions)))
     
    while True:
        try:
            raw_input = input
            useVersion = raw_input("Tip in [y]es or [n]o: ")
            %store useVersion
            
        # Asks if the user wants to load existing data set or wants to start a new query
            
            if useVersion == 'y' or useVersion == 'yes':              
                editorActions_df = pd.read_csv('cache/%i_%s_editorActions.csv' % (userid_input, username_input))
                editorActions_df = editorActions_df.drop(['Unnamed: 0'],axis=1)
                wordsOwned_df = pd.read_csv('cache/%i_%s_wordsOwned.csv' % (userid_input, username_input))
                wordsOwned_df = wordsOwned_df.drop(['Unnamed: 0'],axis=1)
                print('Old dataset successfully loaded!')
                break
                    
            elif useVersion =='n' or useVersion == 'no':
                
                print('You have started a new request. The active date set will be replaced. The process takes a few minutes!')              
                
                getEditorActions = getEditorActions(int(userid_input))
                editorActions_df = getEditorActions[0]
                wordsOwned_df = getEditorActions[1] 
                editorActions_df.to_csv("%s" %feditorActions, sep=',', encoding='utf-8')
                wordsOwned_df.to_csv("%s" %fwordsOwned, sep=',', encoding='utf-8')  
                break
                    
            else:
                continue
        
        except:
            break
     
else:
    print('You have started a new request. There is no active date set. The process takes a few minutes!')
    getEditorActions = getEditorActions(int(userid_input))
    editorActions_df = getEditorActions[0]
    wordsOwned_df = getEditorActions[1] 
    
    editorActions_df.to_csv("%s" %feditorActions, sep=',', encoding='utf-8')
    wordsOwned_df.to_csv("%s" %fwordsOwned, sep=',', encoding='utf-8')

#### This table shows all activities of the author in all articles in which he is involved.

In [ ]:
df_actions1 = pd.DataFrame()
df_actions2 = pd.DataFrame()

df_new = editorActions_df.drop(['ym','article_name','talkpage_entries'],axis=1).groupby('pageid').sum().reset_index()
df_new = df_new.drop(['pageid'],axis=1)
df_new = df_new.dropna()

test = df_new.columns.values.tolist()

for column in df_new:

    if (column == 'adds_survived_ratio' 
        or column == 'dels_survived_ratio' 
        or column == 'reins_survived_ratio' 
        ):
        
        df_column_not_null = df_new[df_new["total_"+str(column.replace("_survived_ratio",""))]>0]
        maxV = df_new[column].max()
        maxV = round(maxV, 2)
        minV = df_new[column].min()
        minV = round(minV, 2)
        meanV = df_new[column].mean()
        meanV = round(meanV, 2)
        
    if (column == 'total_actions_survived_ratio' ):
        df_column_not_null = df_new[df_new[str(column.replace("_survived_ratio",""))]>0]
        maxV = df_new[column].max()
        maxV = round(maxV, 2)
        minV = df_new[column].min()
        minV = round(minV, 2)
        meanV = df_new[column].mean()
        meanV = round(meanV, 2)
        
    else:
        df_column_not_null = df_new[df_new[column]>0] 
        maxV = df_column_not_null[column].max()
        maxV = round(maxV, 2)
        minV = df_column_not_null[column].min()
        minV = round(minV, 2)
        meanV = df_column_not_null[column].mean()
        meanV = round(meanV, 2)
        
    df_actions1 = df_actions1.append({'Max. Value':maxV, 'Min. Value':minV, 'Mean Value (per article)':meanV}, ignore_index=True)

for i in range (0,len(test)):
    name = test[i]
    df_actions2 = df_actions2.append({'Name':name}, ignore_index=True)

result = pd.concat([df_actions1, df_actions2], axis=1)    

source = ColumnDataSource(result)

columns = [
        TableColumn(field="Name", title="Name"),
        TableColumn(field="Min. Value", title="Min. Value"),
        TableColumn(field="Max. Value", title="Max. Value"), 
        TableColumn(field="Mean Value (per article)", title="Mean Value (per article)"),     
    ]
data_table = DataTable(source=source, columns=columns, width=900, height=400)

output_notebook()

show(widgetbox(data_table))

In [ ]:
editorActions_df2 = editorActions_df
%store editorActions_df2
editorActions_Graph = editorActions_df.groupby('ym').sum()
editorActions_Graph = editorActions_Graph.reset_index(drop=False)

### Top Articles by Activity - grouped by article_name

This table shows all activities of the author grouped by article

In [ ]:
editorActions_df2['total_actions_stopword_count'].fillna(0, inplace=True)
source = ColumnDataSource(editorActions_df2.groupby('article_name').sum())

columns = [
        TableColumn(field="article_name", title="Article Name"),
        TableColumn(field="pageid", title="Article ID"),
        TableColumn(field="total_adds", title="total_adds"),
        TableColumn(field="adds_stopword_count", title="adds_stopword_count"),
        TableColumn(field="adds_surv_48h", title="adds_surv_48h"),
        TableColumn(field="adds_survived_ratio", title="adds_survived_ratio"),
        TableColumn(field="total_dels", title="total_dels"),
        TableColumn(field="dels_stopword_count", title="dels_stopword_count"),
        TableColumn(field="dels_surv_48h", title="dels_surv_48h"),
        TableColumn(field="dels_survived_ratio", title="dels_survived_ratio"),
        TableColumn(field="total_reins", title="total_reins"),
        TableColumn(field="reins_stopword_count", title="reins_stopword_count"),
        TableColumn(field="reins_surv_48h", title="reins_surv_48h"),
        TableColumn(field="reins_survived_ratio", title="reins_survived_ratio"),
        TableColumn(field="total_actions", title="total_actions"),
        TableColumn(field="total_actions_stopword_count", title="total_actions_stopword_count"),
        TableColumn(field="total_actions_surv_48h", title="total_actions_surv_48h"),
        TableColumn(field="total_actions_survived_ratio", title="total_actions_survived_ratio"),
        TableColumn(field="pageviews", title="pageviews"),
        TableColumn(field="talkpage_entries", title="talkpage_entries"),  
    ]
data_table = DataTable(source=source, columns=columns, width=1200, height=400)
output_notebook()
show(widgetbox(data_table))

### Activity Overview

In [ ]:
df4 = editorActions_Graph
df4['ym']= pd.to_datetime(df4['ym'])

In [ ]:
begin = list(df4['ym'])
end = sorted(begin, reverse=True)

In [ ]:
def actitivtyOverview(begin,end):
    
    df4 = editorActions_Graph
    df4['ym']= pd.to_datetime(df4['ym'])
    
    df4 = df4[(df4.ym >= begin) &(df4.ym <= end)]

    trace1 = go.Scatter(x = list(df4['ym']),y = list(df4.total_adds),
        mode = 'lines+markers',name = 'Adds',
        marker = dict(color = 'rgba(0, 128, 43, .8)'))

    trace2 = go.Scatter(
        x = list(df4['ym']),y = list(df4.total_dels),
        mode = 'lines+markers',name = 'Dels',
        marker = dict(color = 'rgba(255, 0, 0, .8)'))

    trace3 = go.Scatter(
        x = list(df4['ym']),y = list(df4.total_reins),
        mode = 'lines+markers',name = 'Reins',
        marker = dict(color = 'rgba(0, 51, 153, .8)'))

    layout= go.Layout(hovermode= 'closest',
        xaxis= dict(title= 'Month',ticklen= 5,zeroline= True,gridwidth= 2),
        yaxis=dict(title= 'Actions',ticklen= 5,gridwidth= 2),
        showlegend= True)

    data = [trace1,trace2,trace3]

    plotly.offline.init_notebook_mode(connected=True)
    plotly.offline.iplot({"data": data,"layout": layout})
    
a = interact(actitivtyOverview,begin=begin,end=end)

### Adds Survival Ratio

In [ ]:
def addRatio(begin,end):
    
    df4 = editorActions_Graph
    df4['ym']= pd.to_datetime(df4['ym'])
    
    df4 = df4[(df4.ym >= begin) &(df4.ym <= end)]


    trace1 = go.Bar(
        x=list(df4['ym']),y=list(df4.total_adds),
        name='Total adds',
        marker = dict(color = 'rgba(0, 128, 43, .8)'))

    trace2 = go.Bar(
        x=list(df4['ym']),y=list(df4.adds_surv_48h),
        name='48h survived',
        marker = dict(color = 'rgba(0, 230, 77, .8)'))

    data = [trace1, trace2]

    layout= go.Layout(hovermode= 'closest',
        xaxis= dict(title= 'Month',ticklen= 5,zeroline= True,gridwidth= 2),
        yaxis=dict(title= 'Actions',ticklen= 5,gridwidth= 2),
        showlegend= True,barmode='group')

    plotly.offline.init_notebook_mode(connected=True)
    plotly.offline.iplot({"data": data,"layout": layout})

b = interact(addRatio,begin=begin,end=end)

### Deletes Survival Ratio

In [ ]:
def delRatio(begin,end):
    
    df4 = editorActions_Graph
    df4['ym']= pd.to_datetime(df4['ym'])
    
    df4 = df4[(df4.ym >= begin) &(df4.ym <= end)]

    trace1 = go.Bar(
        x=list(df4['ym']),y=list(df4.total_dels),
        name='Total dels',
        marker = dict(color = 'rgba(255, 0, 0, .8)'))

    trace2 = go.Bar(
        x=list(df4['ym']),y=list(df4.dels_surv_48h),
        name='48h survived',
        marker = dict(color = 'rgba(255, 102, 102, .8)'))

    layout= go.Layout(hovermode= 'closest',
        xaxis= dict(title= 'Month',ticklen= 5,zeroline= True,gridwidth= 2),
        yaxis=dict(title= 'Actions',ticklen= 5,gridwidth= 2),
        showlegend= True,barmode='group')

    data = [trace1, trace2]

    plotly.offline.init_notebook_mode(connected=True)
    plotly.offline.iplot({"data": data,"layout": layout})

c = interact(delRatio,begin=begin,end=end)

### Reinserts Survival Ratio

In [ ]:
def reinsRatio(begin,end):
    
    df4 = editorActions_Graph
    df4['ym']= pd.to_datetime(df4['ym'])
    
    df4 = df4[(df4.ym >= begin) &(df4.ym <= end)]

    trace1 = go.Bar(
        x=list(df4['ym']),y=list(df4.total_reins),
        name='Total reinserts',
        marker = dict(color = 'rgba(0, 51, 153, .8)'))

    trace2 = go.Bar(
        x=list(df4['ym']),y=list(df4.reins_surv_48h),
        name='48h survived',
        marker = dict(color = 'rgba(0, 153, 255, .8)'))

    data = [trace1, trace2]

    layout= go.Layout(hovermode= 'closest',
        xaxis= dict(title= 'Month',ticklen= 5,zeroline= True,gridwidth= 2),
        yaxis=dict(title= 'Actions',ticklen= 5,gridwidth= 2),
        showlegend= True,barmode='group')

    plotly.offline.init_notebook_mode(connected=True)
    plotly.offline.iplot({"data": data,"layout": layout})

d = interact(reinsRatio,begin=begin,end=end)

### Articles with most owned words [%]

The subsequent Table shows only the last revision of the articles

In [ ]:
%store wordsOwned_df

pageview_groupby_page_df = editorActions_df2.groupby('pageid').agg({'pageviews': 'sum'}).reset_index()
wordsOwned_pageviews_df = pd.merge(left=wordsOwned_df,right=pageview_groupby_page_df, how='left', left_on=['pageid'], right_on=['pageid'])#.drop(['title'], axis=1) 

source = ColumnDataSource(wordsOwned_pageviews_df)

columns = [
        TableColumn(field="article_name", title="Article Name"),
        TableColumn(field="pageid", title="Article ID"),
        TableColumn(field="words_owned", title="Words owned"),
        TableColumn(field="words_owned_ratio", title="Ratio (percent of words user owns in latest revision)"),
        TableColumn(field="pageviews", title="pageviews") ]
data_table = DataTable(source=source, columns=columns, width=800, height=300)
output_notebook()
show(widgetbox(data_table))

### Articles with most Actions per Edit and Time Spent Editing [%]

In [ ]:
from metrics.getTopXArticles_Actions_Timespent import getTopXArticles_Actions_Timespent
Actions_Timespent = 'cache/%i_%s_Actions_Timespent.csv' % (userid_input, username_input)

if os.path.isfile(Actions_Timespent)== True:
     
    if useVersion == 'y' or useVersion == 'yes':              
        Actions_Timespent_df = pd.read_csv(Actions_Timespent)
        Actions_Timespent_df = Actions_Timespent_df.drop(['Unnamed: 0'],axis=1)
        %store Actions_Timespent_df
        print('Old dataset successfully loaded!')
                    
    elif useVersion =='n' or useVersion == 'no':
        print('You have started a new request. The active date set will be replaced. The process takes a few minutes!')              
        Actions_Timespent_df = getTopXArticles_Actions_Timespent(str(userid_input))
        %store Actions_Timespent_df
        Actions_Timespent_df.to_csv("%s" %Actions_Timespent, sep=',', encoding='utf-8')
    else:
        print('can not be the case')
    
else:

    print('You have started a new request. There is no active date set. The process takes a few minutes!')
    Actions_Timespent_df = getTopXArticles_Actions_Timespent(str(userid_input))
    %store Actions_Timespent_df
    Actions_Timespent_df.to_csv("%s" %Actions_Timespent, sep=',', encoding='utf-8')

In [ ]:
source = ColumnDataSource(Actions_Timespent_df)

columns = [
        TableColumn(field="title", title="Articlename"),
        TableColumn(field="ActionsperEditsWithinHour", title="ActionsperEditsWithinHour"),
        TableColumn(field="ActionsperEditsAfterHour", title="ActionsperEditsAfterHour"),
        TableColumn(field="ActionsperEditsExactlyinHour", title="ActionsperEditsExactlyinHour"),
        TableColumn(field="SessionsWithinHour", title="SessionsWithinHour"),
        TableColumn(field="SessionsAfterHour", title="SessionsAfterHour"),
        TableColumn(field="SessionsExactlyinHour", title="SessionsExactlyinHour")]
data_table = DataTable(source=source, columns=columns, width=800, height=300)
output_notebook()
show(widgetbox(data_table))

In [ ]:
table = [
    ['Avg. Actions per Edits Within Hour: ', Actions_Timespent_df.ActionsperEditsWithinHour.mean()],
    ['Avg. Actions per Edits After one Hour: ',Actions_Timespent_df.ActionsperEditsAfterHour.mean()],
    ['Avg. Actions per Edits Exactly in one Hour: ',Actions_Timespent_df.ActionsperEditsExactlyinHour.mean()],
    ['Avg. Sessions Within one Hour: ',Actions_Timespent_df.SessionsWithinHour.mean()],
    ['Avg. Sessions After one Hour: ',Actions_Timespent_df.SessionsAfterHour.mean()],
    ['Avg. Sessions Exactly in one Hour: ',Actions_Timespent_df.SessionsExactlyinHour.mean()]
       ]

display(HTML(tabulate.tabulate(table, tablefmt='html')))

### Articles with the highest Conflict Rate
Conflict rate is compute as follows :

For a given article: A and editor: X, if X added/deleted a token and another editor B del/add the same token after X and X add/del it after B again, it will be count as a conflict. 

For computing the conflict score, the following code is used

t1 = difference between add/del1 by A and del/add2 by B in seconds <br>
t2 = difference between add/del2 by B and del/add3 by A in seconds <br>
$c_1=\frac{\lg3600}{\lg(t_1 + 2.0)}$ <br>

$c_2=\frac{\lg3600}{\lg(t_2 + 2.0)}$<br>

S = Sum of all c1 and c2 over a token <br>
A = Actions of A in this token <br>

$conflict score = \frac{S}{A}$<br>

this returns a value between 0 and 7.5, and the value diverges to 0, the longer the time between add / del, so changes for under 1 hour are weighted stronger.


In [ ]:
from metrics.getTopXArticles_by_conflict_score import getTopXArticles_by_conflict_score
TopConflictScore = 'cache/%i_%s_TopConflictScore.csv' % (userid_input, username_input)


if os.path.isfile(TopConflictScore)== True:
     
    if useVersion == 'y' or useVersion == 'yes':              
        TopConflictScore_df = pd.read_csv(TopConflictScore)
        TopConflictScore_df = TopConflictScore_df.drop(['Unnamed: 0'],axis=1)
        %store TopConflictScore_df
        print('Old dataset successfully loaded!')
                    
    elif useVersion =='n' or useVersion == 'no':
        print('You have started a new request. The active date set will be replaced. The process takes a few minutes!')              
        TopConflictScore_df = getTopXArticles_by_conflict_score(userid_input)
        %store TopConflictScore_df
        TopConflictScore_df.to_csv("%s" %TopConflictScore, sep=',', encoding='utf-8')
    else:
        print('can not be the case')
    
else:

    print('You have started a new request. There is no active date set. The process takes a few minutes!')
    TopConflictScore_df = getTopXArticles_by_conflict_score(userid_input)
    %store TopConflictScore_df
    TopConflictScore_df.to_csv("%s" %TopConflictScore, sep=',', encoding='utf-8')

In [ ]:
source = ColumnDataSource(TopConflictScore_df)

columns = [
        TableColumn(field="title", title="Articlename"),
        TableColumn(field="conflict_score", title="Conflict Score")]
data_table = DataTable(source=source, columns=columns, width=800, height=300)
output_notebook()
show(widgetbox(data_table))

### Select an article and see further information
Please choose an article and click on see statistics

In [ ]:
from ipywidgets import widgets
from IPython.display import display

optionList = list(TopConflictScore_df.title)
optionList = sorted(optionList)

# collect user articles in a list and display it

chooseArticle = widgets.Select(options=optionList,value=optionList[0],description='Articles: ',disabled=False)
display(chooseArticle)

df = TopConflictScore_df

button = widgets.Button(description="Choose!")
display(button)

def on_button_clicked(b):
    global article_name
    article_name = chooseArticle.value
    article_name = article_name.replace(" ", "_")
    % store article_name

    global article_id
    article_id = df.loc[df['title'] == chooseArticle.value].pageid.iloc[0]
    article_id = int(article_id)
    % store article_id
    
button.on_click(on_button_clicked)

In [ ]:
%%html
<a href=notebook3.ipynb target="_blank">Go to Workbook three</a>